In [26]:
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain_groq import ChatGroq
import streamlit as st

In [27]:
from pinecone import Pinecone,ServerlessSpec
load_dotenv()
pc=Pinecone(api_key="68234ea3-63f7-4ebf-99db-9f5fdecca789")


In [28]:
index_name = "sadat"
if index_name not in pc.list_indexes().names():
  pc.create_index(
    name="sadat",
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
      cloud="aws",
      region="us-east-1"
    ),
    
  )

In [29]:
def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)

    documents=loader.load()
    return documents



In [30]:
##extracting data
data_ext=load_pdf("data/")

In [31]:
##splitting all docs
def text_split(data_ext):
    splited_text=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=splited_text.split_documents(data_ext)
    return text_chunks


In [32]:
text_chunks=text_split(data_ext)

In [33]:
## download model for Embedding
def hf_embeddings():
    embedding=HuggingFaceEmbeddings(model_name = ("sentence-transformers/all-MiniLM-L6-v2"))
    return embedding

In [34]:
embeddings=hf_embeddings()

c:\Users\arshu\anaconda3\envs\chatbot\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [35]:
import os

if index_name not in pc.list_indexes().names():
    os.environ['PINECONE_API_KEY'] = '68234ea3-63f7-4ebf-99db-9f5fdecca789'
    index_name="sadat"

    docsearch=PineconeVectorStore.from_documents(text_chunks,embeddings,index_name=index_name)
os.environ['PINECONE_API_KEY'] = '68234ea3-63f7-4ebf-99db-9f5fdecca789'
docsearch=PineconeVectorStore.from_existing_index(index_name,embeddings)


In [36]:
docsearch=PineconeVectorStore.from_existing_index(index_name,embeddings)
question="cancer"
docs=docsearch.similarity_search(question,k=3)
print('result: ',docs)


result:  [Document(id='df5e6e9b-f9a4-42a6-ba13-aa56f36c36b9', metadata={'page': 591.0, 'source': 'data\\Medical_book.pdf'}, page_content='cancer tospread to other organs of the body.'), Document(id='8a6d90d8-78a1-4740-89f5-bb249ad66d55', metadata={'page': 43.0, 'source': 'data\\Medical_book.pdf'}, page_content='and other tumors.'), Document(id='ac08b54e-bb3c-4d85-a7cb-d1ce453843b2', metadata={'page': 593.0, 'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 580Breast cancerGEM -0433 to 0624 - B  10/22/03 6:09 PM  Page 580')]


In [37]:
load_dotenv()
import os
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [39]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [40]:
from langchain.chains import RetrievalQA

qa=RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=docsearch.as_retriever(search_kwargs={'k':2}),
        return_source_documents=True,
        )

In [46]:
user_input=st.text_input("What is your Question")
if user_input:
    result=qa.invoke({"query": user_input})
    st.write("Response:",result["result"])
    print("Response: ", result["result"])

2024-10-07 21:36:04.393 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 21:36:04.393 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 21:36:04.394 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 21:36:04.395 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 21:36:04.396 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-07 21:36:04.396 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
